In [ ]:
from ollama import Client

# model = "qwen2.5:0.5b"
# model = "ollama3"
model = "deepseek-r1:1.5b"
client = Client(host="http://localhost:11434")

client.ps()

ProcessResponse(models=[Model(model='deepseek-r1:1.5b', name='deepseek-r1:1.5b', digest='e0979632db5a88d1a53884cb2a941772d10ff5d055aabaa6801c4e36f3a6c2d7', expires_at=datetime.datetime(2025, 6, 8, 22, 16, 53, 799693, tzinfo=TzInfo(UTC)), size=2046867968, size_vram=2046867968, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='1.8B', quantization_level='Q4_K_M'))])

In [ ]:

import json
from pydantic import BaseModel


query = "ONLY give me description of an ice cream that is pink color"

In [74]:
request = type("object", (object,), {})()
request.query = query


item = type("object", (object,), {})()
item.description = """
https://en.wikipedia.org/wiki/Superman_and_the_Mole_Men	{"@type": "Movie", "@id": "https://www.wikidata.org/wiki/Q1115677", "name": "Superman and the Mole Men", "datePublished": 1951, "director": {"@type": "Person", "name": "Lee Sholem", "@id": "http://www.wikidata.org/entity/Q6515055"}, "image": "https://commons.wikimedia.org/wiki/Special:FilePath/1951Superman002.jpg?width=800", "inLanguage": "English", "url": "https://en.wikipedia.org/wiki/Superman_and_the_Mole_Men", "description": "Superman and the Mole Men (titled onscreen as Superman and the Mole-Men, but all posters and advertising omit the hyphen) is a 1951 American independent black-and-white superhero film released by Lippert Pictures. Produced by Barney A. Sarecky and directed by Lee Sholem, it stars George Reeves as Superman and Phyllis Coates as Lois Lane. It is the first feature film based on any DC Comics character, and the first time George Reeves played the role of Superman. In the film's story, reporters Clark Kent and Lois Lane arrive in the small town of Silsby to witness the drilling of the world's deepest oil well. The drill, however, has penetrated the underground home of a race of small, bald humanoids who, out of curiosity, climb to the surface at night. They glow in the dark, which scares the local townspeople who form a mob intent on killing the creatures. Only Superman can intervene to prevent tragedy.", "actor": [{"@type": "Person", "@id": "http://www.wikidata.org/entity/Q3565867", "name": "Walter Reed"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q3496795", "name": "Stanley Andrews"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q342537", "name": "George Reeves", "characterName": "Superman"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q448430", "name": "Jeff Corey"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q1307857", "name": "Frank Reicher"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q3177488", "name": "Jerry Maren"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q3038280", "name": "J. Farrell MacDonald"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q5004353", "name": "Byron Foulger"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q1311138", "name": "Phyllis Coates", "characterName": "Lois Lane"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q863063", "name": "Billy Curtis"}, {"@type": "Person", "@id": "http://www.wikidata.org/entity/Q15069891", "name": "Ray Walker"}]}
"""


class DetectItemTypePrompt(BaseModel):
    item_type: str


class RankingPrompt(BaseModel):
    score: int
    description: str


all_prompts = {
    "DetectItemTypePrompt": {
        "prompt": f"""What is the kind of item the query is likely seeking with this query: {request.query}""",
        "schema": {"item_type": ""},
        "type_schema": DetectItemTypePrompt,
    },
    "RankingPrompt": {
        "prompt": f"""
        Assign a score between 0 and 100 to the following item
        based on how relevant it is to the user's question. Use your knowledge from other sources, about the item, to make a judgement. 
        If the score is above 50, provide a short description of the item highlighting the relevance to the user's question, without mentioning the user's question.
        Provide an explanation of the relevance of the item to the user's question, without mentioning the user's question or the score or explicitly mentioning the term relevance.
        If the score is below 75, in the description, include the reason why it is still relevant.
        The user's question is: \"{request.query}\". The item's description in schema.org format is \"{item.description}\".
    """,
        "schema": {
            "score": "integer between 0 and 100",
            "description": "short description of the item",
        },
        "type_schema": RankingPrompt,
    },
}

current_prompt = all_prompts["RankingPrompt"]

prompt = current_prompt["prompt"]
schema = current_prompt["schema"]
system_prompt = f"""You are a helpful assistant that always provides responses in JSON format.
Your response must be valid JSON that matches this schema: {json.dumps(schema)}
Only output the JSON object, no additional text or explanation."""
type_schema = current_prompt["type_schema"].model_json_schema()

print(type_schema)

{'properties': {'score': {'title': 'Score', 'type': 'integer'}, 'description': {'title': 'Description', 'type': 'string'}}, 'required': ['score', 'description'], 'title': 'RankingPrompt', 'type': 'object'}


In [ ]:

# system_prompt = "You are a helpful assistant"
# prompt = "hi what is your name?"
# print(f"Asking: {prompt}")
response = client.chat(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ],
    model=model,
    options={
        "temperature": .7,
    },
    format={'properties': {'score': {'title': 'Score', 'type': 'integer'}, 'description': {'title': 'Description', 'type': 'string'}}, 'required': ['score', 'description'], 'title': 'RankingPrompt', 'type': 'object'}
)

response

ChatResponse(model='deepseek-r1:1.5b', created_at='2025-06-08T22:54:20.379009991Z', done=True, done_reason='stop', total_duration=3864011952, load_duration=296078743, prompt_eval_count=1007, prompt_eval_duration=12816976, eval_count=66, eval_duration=3546529680, message=Message(role='assistant', content='{\n       "score": 75,\n       "description": "The item is a movie description that provides information about the movie\'s title, release date, director, actors, genre, and other details. It is not directly relevant to the user\'s question which is about an ice cream color in pink."\n     }\n    ', thinking=None, images=None, tool_calls=None))